In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from skimpy import clean_columns

In [36]:
landscape_cat = {
    "ag": [
        "Corn",
        "Cotton",
        "Rice",
        "Sorghum",
        "Soybeans",
        "Sunflower",
        "Peanuts",
        "Tobacco",
        "Sweet Corn",
        "Pop or Orn Corn",
        "Mint",
        "Barley",
        "Durum Wheat",
        "Spring Wheat",
        "Winter Wheat",
        "Other Small Grains",
        "Dbl Crop WinWht/Soybeans",
        "Rye",
        "Oats",
        "Millet",
        "Speltz",
        "Canola",
        "Flaxseed",
        "Safflower",
        "Rape Seed",
        "Mustard",
        "Alfalfa",
        "Other Hay/Non Alflafa",
        "Camelina",
        "Buckwheat",
        "Sugarbeets",
        "Dry Beans",
        "Potatoes",
        "Other Crops",
        "Sugarcane",
        "Sweet Potatoes",
        "Misc Vegs & Fruits",
        "Watermelons",
        "Onions",
        "Cucumbers",
        "Chick Peas",
        "Lentils",
        "Peas",
        "Tomatoes",
        "Caneberries",
        "Hops",
        "Herbs",
        "Clover/Wildflowers",
        "Sod/Grass Seed",
        "Switchgrass",
        "Cherries",
        "Peaches",
        "Apples",
        "Grapes",
        "Christmas Trees",
        "Other Tree Crops",
        "Citrus",
        "Pecans",
        "Almonds",
        "Walnuts",
        "Pears",
        "Pistachios",
        "Triticale",
        "Carrots",
        "Asparagus",
        "Garlic",
        "Cantaloupes",
        "Prunes",
        "Olives",
        "Oranges",
        "Honeydew Melons",
        "Broccoli",
        "Avocados",
        "Peppers",
        "Pomegranates",
        "Nectarines",
        "Greens",
        "Plums",
        "Strawberries",
        "Squash",
        "Apricots",
        "Vetch",
        "Dbl Crop WinWht/Corn",
        "Dbl Crop Oats/Corn",
        "Lettuce",
        "Dbl Crop Triticale/Corn",
        "Pumpkins",
        "Dbl Crop Lettuce/Durum Wht",
        "Dbl Crop Lettuce/Cantaloupe",
        "Dbl Crop Lettuce/Cotton",
        "Dbl Crop Lettuce/Barley",
        "Dbl Crop Durum Wht/Sorghum",
        "Dbl Crop Barley/Sorghum",
        "Dbl Crop WinWht/Sorghum",
        "Dbl Crop Barley/Corn",
        "Dbl Crop WinWht/Cotton",
        "Dbl Crop Soybeans/Cotton",
        "Dbl Crop Soybeans/Oats",
        "Dbl Crop Corn/Soybeans",
        "Blueberries",
        "Cabbage",
        "Cauliflower",
        "Celery",
        "Radishes",
        "Turnips",
        "Eggplants",
        "Gourds",
        "Cranberries",
        "Dbl Crop Barley/Soybeans",
        "Cropland",
    ],
    "nat": [
        "Forest",
        "Shrubland",
        "Barren",
        "Wetlands",
        "Nonag/Undefined",
        "Deciduous Forest",
        "Evergreen Forest",
        "Mixed Forest",
        "Shrubland",
        "Woody Wetlands",
        "Herbaceous Wetlands",
    ],
    "semi_nat": ["Grassland/Pasture", "Grass/Pasture"],
    "dev": [
        "Developed/Open Space",
        "Developed/Low Intensity",
        "Developed/Med Intensity",
        "Developed/High Intensity",
        "Barren",
    ],
    "other": ["Aquaculture", "Water", "Perennial Ice/Snow", "Open Water"],
}


In [37]:
def cdl_processing(cdl_metadata, qgis_csv):

    cdl_metadata["cdl_number"] = "VALUE_" + cdl_metadata["cdl_number"].astype("str")
    qgis_csv.rename(
        columns=cdl_metadata.set_index("cdl_number")["cdl_classification"], inplace=True
    )
    qgis_csv = qgis_csv.drop(
        [
            "descriptio",
            "timestamp",
            "begin",
            "end",
            "altitudeMo",
            "tessellate",
            "extrude",
            "visibility",
            "drawOrder",
            "icon",
        ],
        axis=1,
    )

    qgis_csv["Total"] = qgis_csv.sum(axis=1, numeric_only=True)
    qgis_csv['Ag'] = qgis_csv[qgis_csv.columns.intersection(landscape_cat['ag'])].sum(axis=1)
    qgis_csv["Ag_Prop"] = qgis_csv["Ag"] / qgis_csv["Total"]
    qgis_csv['Nat'] = qgis_csv[qgis_csv.columns.intersection(landscape_cat['nat'])].sum(axis=1)
    qgis_csv["Nat_Prop"] = qgis_csv["Nat"] / qgis_csv["Total"]
    qgis_csv['Semi_Nat'] = qgis_csv[qgis_csv.columns.intersection(landscape_cat['semi_nat'])].sum(axis=1)
    qgis_csv["Semi_Nat_Prop"] = qgis_csv["Semi_Nat"] / qgis_csv["Total"]
    qgis_csv['Dev'] = qgis_csv[qgis_csv.columns.intersection(landscape_cat['dev'])].sum(axis=1)
    qgis_csv["Dev_Prop"] = qgis_csv["Dev"] / qgis_csv["Total"]
    qgis_csv['Other'] = qgis_csv[qgis_csv.columns.intersection(landscape_cat['other'])].sum(axis=1)
    qgis_csv["Other_Prop"] = qgis_csv["Other"] / qgis_csv["Total"]
    #"] / qgis_csv["Total"]
    #qgis_csv["Soybean_Prop"] = qgis_csv["Soybeans"] / qgis_csv["Total"]
    #qgis_csv["Wheat_Prop"] = qgis_csv["Winter Wheat"] / qgis_csv["Total"]
    qgis_csv["Forest_Prop"] = (qgis_csv.filter(like='Forest').sum(axis=1)) / qgis_csv["Total"]
    qgis_csv["Total_Nat_Prop"] = qgis_csv["Nat_Prop"] + qgis_csv["Semi_Nat_Prop"]

    return qgis_csv


In [38]:
def calc_props(year, state, buffer):
    metadata = pd.read_csv("Metadata/cdl_metadata_" + str(state) + "_" + str(year) + ".csv")
    cdl_df = pd.read_csv("QGIS/QGIS_CSV/" + str(year) + "/" + state.upper() + "_" + str(buffer) + "m.csv")
    cdl_df = cdl_processing(metadata, cdl_df)
    cdl_df["Buffer"] = buffer
    cdl_df.to_csv("CDL/" + str(year) + "_" + str(state) + "_cdl_" + str(buffer) + "m.csv")

In [40]:
calc_props(2023, "ny", 100)
calc_props(2023, "ny", 250)
calc_props(2023, "ny", 500)
calc_props(2023, "ny", 1000)

In [42]:
calc_props(2023, "nj", 100)
calc_props(2023, "nj", 250)
calc_props(2023, "nj", 500)
calc_props(2023, "nj", 1000)

In [43]:
calc_props(2023, "me", 100)
calc_props(2023, "me", 250)
calc_props(2023, "me", 500)
calc_props(2023, "me", 1000)